In [ ]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
import os
import pytorch_lightning as pl
import os 
from torch.optim import Adadelta


import torchvision.transforms.v2 as v2
import torch
from torchvision.transforms import Compose, ToTensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import warnings

import utils
import utils_mgr
from utils_mgr import getAudio, DataAudio

import pickle



In [ ]:

def import_and_preprocess_data(archtecture_type = "1D"):
    # Load metadata and features.
    tracks = utils.load('data/fma_metadata/tracks.csv')


    #Select the desired subset among the entire dataset
    sub = 'small'
    raw_subset = tracks[tracks['set', 'subset'] <= sub] 
    
    #Creation of clean subset for the generation of training, test and validation sets
    meta_subset= utils_mgr.create_subset(raw_subset)

    # Remove corrupted files
    corrupted = [98565, 98567, 98569, 99134, 108925, 133297]
    meta_subset = meta_subset[~meta_subset['index'].isin(corrupted)]

    #Split between taining, validation and test set according to original FMA split

    train_set = meta_subset[meta_subset["split"] == "training"]
    val_set   = meta_subset[meta_subset["split"] == "validation"]
    test_set  = meta_subset[meta_subset["split"] == "test"]

    # Standard transformations for images
    # Mean and std are computed on one file of the training set
    transforms = v2.Compose([torch.Tensor,
                            lambda x: x/0.21469156])  #To normalize data

    # Create the datasets and the dataloaders
    """
    train_dataset    = DataAudio(train_set, transform = transforms, type=archtecture_type)
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)

    val_dataset      = DataAudio(val_set, transform = transforms, type=archtecture_type)
    val_dataloader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)

    """
    test_dataset     = DataAudio(test_set, transform = transforms, type=archtecture_type)
    test_dataloader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

    return test_dataloader